In [1]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter
import syft as sy 

In [2]:
torch.manual_seed(42)
hook = sy.TorchHook(torch)

In [3]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [4]:
wine_data = pd.read_csv('../data/winequality-white.csv',sep=';')
wine_data = wine_data.values
x = wine_data[:,:11]
x_s = preprocessing.StandardScaler().fit(x)
x = x_s.transform(x)
y = wine_data[:,11]

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3)
X = torch.from_numpy(x_train).type(torch.FloatTensor)
Y = torch.from_numpy(y_train).type(torch.FloatTensor)
X_t = torch.from_numpy(x_test).type(torch.FloatTensor)
Y_t = torch.from_numpy(y_test).type(torch.FloatTensor)

In [6]:
print(len(X), len(Y), len(X_t), len(Y_t))

3428 3428 1470 1470


In [7]:
BREAKING_TRAIN = 2800
BREAKING_TEST = 1200
bob_x = X[:BREAKING_TRAIN].send(bob)
bob_y = Y[:BREAKING_TRAIN].send(bob)
bob_x_t = X_t[:BREAKING_TEST].send(bob)
bob_y_t = Y_t[:BREAKING_TEST].send(bob)

alice_x = X[BREAKING_TRAIN:].send(alice)
alice_y = Y[BREAKING_TRAIN:].send(alice)
alice_x_t = X_t[BREAKING_TEST:].send(alice)
alice_y_t = Y_t[BREAKING_TEST:].send(alice)

data_loader = [(bob_x, bob_y), (alice_x, alice_y)]
test_loader = [(bob_x_t, bob_y_t), (alice_x_t, alice_y_t)]

In [8]:
for data, target in test_loader:
    print(data.location, target.location)

<VirtualWorker id:bob #objects:4> <VirtualWorker id:bob #objects:4>
<VirtualWorker id:alice #objects:4> <VirtualWorker id:alice #objects:4>


In [9]:
class SVR(nn.Module):
    def __init__(self):
        super(SVR,self).__init__()
        self.fc1 = nn.Linear(11, 1)
        self.fc2 = nn.Linear(1, 1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

In [10]:
model = SVR()
criterion = nn.MSELoss()

In [11]:
TOTAL_EPOCH = 1200

for epoch in range(TOTAL_EPOCH):
    for data, target in data_loader:
        model.send(data.location)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        model = model.get()
    
    if epoch % 100 == 0:
        print('loss: {}'.format(loss.get()))

loss: 31.162527084350586


/home/zfj/anaconda3/envs/torch1.4/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/native.py:418: UserWarning: Using a target size (torch.Size([2800])) that is different to the input size (torch.Size([2800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  response = command_method(*args_, **kwargs_)
/home/zfj/anaconda3/envs/torch1.4/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/native.py:418: UserWarning: Using a target size (torch.Size([628])) that is different to the input size (torch.Size([628, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  response = command_method(*args_, **kwargs_)


loss: 25.75701904296875
loss: 20.52824592590332
loss: 16.099811553955078
loss: 13.294962882995605
loss: 10.524943351745605
loss: 7.323026657104492
loss: 3.861070394515991
loss: 1.7866584062576294
loss: 0.8747089505195618
loss: 0.7508062124252319
loss: 0.7508059740066528


In [12]:
# bob:alice 1:1 split epoch 1200 loss:0.7886 accuracy: 0.437 for bob, 0.468 fro alice 
# bob:alice 1:4 split epoch 1500 loss:0.7582 accuracy: 0.446 for bob, 0.433 fro alice 
# bob:alice 4:1 split epoch 1200 loss:0.7508 accuracy: 0.47 for bob, 0.44 fro alice 

In [13]:
model.location

In [14]:
def calc_acc_torch(predicted, real):
    pre_round = torch.round(predicted)
    pr_squeezed = torch.squeeze(pre_round)
    correct = (pr_squeezed == real).sum().get().item()
    accuracy = correct/len(real)*1.0
    return accuracy

In [15]:
with torch.no_grad():
    for data, target in test_loader:
        model.send(data.location)
        output = model(data)
        model = model.get()
        print('test_score: {}\n'.format(calc_acc_torch(output, target)))

test_score: 0.47

test_score: 0.4

